In [3]:
import os
import math
import random
import IPython.display as ipd
import numpy as np
import pandas as pd
import keras
import librosa
from keras.models import Sequential
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout
from keras.utils import np_utils
from keras.regularizers import l2
import keras.utils
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import tensorflow as tf
import utils
from keras.models import Sequential
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D)
from keras import optimizers


In [4]:
AUDIO_DIR = os.environ.get('AUDIO_DIR')

tracks = utils.load('data/fma_metadata/tracks.csv')
features = utils.load('data/fma_metadata/features.csv')
# echonest = utils.load('data/fma_metadata/echonest.csv')

np.testing.assert_array_equal(features.index, tracks.index)
# assert echonest.index.isin(tracks.index).all()

tracks.shape, features.shape
#106574 trackss

((106574, 52), (106574, 518))

In [7]:
medium = tracks['set', 'subset'] <= 'medium'

train = tracks['set', 'split'] == 'training'
val = tracks['set', 'split'] == 'validation'
test = tracks['set', 'split'] == 'test'

y_train = tracks.loc[medium & train, ('track', 'genre_top')].to_frame().to_numpy()
y_test = tracks.loc[medium & test, ('track', 'genre_top')].to_frame().to_numpy()
X_train = features.loc[medium & train, 'mfcc'].to_numpy()
X_test = features.loc[medium & test, 'mfcc'].to_numpy()

numFeatures = X_train.shape[1]
Labels = np.unique(y_train)

label_encoder = sklearn.preprocessing.LabelEncoder()
onehot_encoder = sklearn.preprocessing.OneHotEncoder(sparse = False)

integer_encoded_y_train = label_encoder.fit_transform(y_train.reshape(1, y_train.shape[0])[0])
integer_encoded_y_train = integer_encoded_y_train.reshape(len(integer_encoded_y_train), 1)
onehot_encoded_y_train = onehot_encoder.fit_transform(integer_encoded_y_train)

integer_encoded_y_test = label_encoder.fit_transform(y_test.reshape(1, y_test.shape[0])[0])
integer_encoded_y_test = integer_encoded_y_test.reshape(len(integer_encoded_y_test), 1)
onehot_encoded_y_test = onehot_encoder.fit_transform(integer_encoded_y_test)

y_train = onehot_encoded_y_train
y_test = onehot_encoded_y_test


# print(y_train)
# print(y_test)
# print(numFeatures)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
140


In [9]:
def get_model_CNN():
    model = tf.keras.Sequential()
    model.add(Conv2D(2, 2, input_shape = X_train.shape[1:]))
    model.add(Conv2D(1,1, input_shape = (X_train.shape[1], X_train.shape[2])))
    model.add(Conv2D(48, kernel_size=(2, 2), kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    
    model.add(Flatten())
    model.add(tf.keras.layers.Dense(y_train.shape[1], activation = 'softmax'))
    
    return model

In [ ]:
X_train = np.load("/Users/josephzhang/Developer/fma/npy_train.npy") # change directory
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
# print(X_train[1])
# print(X_train.shape)

X_test = np.load("/Users/josephzhang/Developer/fma/npy_test.npy") #change directory
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
model = get_model_CNN()


splitValue = math.ceil(19922 * 0.8)
X_trainFormatted = X_train[:19922]
y_trainFormatted = y_train[:19922]
X_trainFormatted, y_trainFormatted = sklearn.utils.shuffle(X_trainFormatted, y_trainFormatted)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.fit(X_trainFormatted[:splitValue],y_trainFormatted[:splitValue], epochs = 20, validation_data = (X_train[splitValue: splitValue + 3000],y_train[splitValue: splitValue + 3000]))

[[[-5.21129211e+02]
  [-2.47567795e+02]
  [-1.08022316e+02]
  ...
  [-7.11279297e+01]
  [-7.75416031e+01]
  [-9.73930130e+01]]

 [[ 1.55325842e+00]
  [ 8.51672058e+01]
  [ 1.07385818e+02]
  ...
  [ 1.32055450e+02]
  [ 1.35444305e+02]
  [ 1.26504570e+02]]

 [[-8.50553870e-01]
  [-4.24489403e+01]
  [-5.64110718e+01]
  ...
  [-6.07286072e+01]
  [-4.87116470e+01]
  [-3.68379822e+01]]

 ...

 [[ 1.12130344e-02]
  [ 7.66306114e+00]
  [ 2.67736793e-01]
  ...
  [ 3.66442204e+00]
  [ 6.18414736e+00]
  [ 9.55699921e+00]]

 [[ 2.06057692e+00]
  [ 2.22052727e+01]
  [ 2.16350021e+01]
  ...
  [ 1.92207205e+00]
  [ 4.77948475e+00]
  [ 5.36637592e+00]]

 [[ 1.23948824e+00]
  [ 1.19747221e-01]
  [-7.38131225e-01]
  ...
  [-1.29854889e+01]
  [-1.72685471e+01]
  [-1.83781319e+01]]]
(19978, 20, 385, 1)
Epoch 1/20
324/499 [==================>...........] - ETA: 20s - loss: 91.5957 - accuracy: 0.1800